In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob, shutil, os, random, pathlib, pprint, scipy, umap
import seaborn as sns; sns.set()
import random

from tensorflow import keras
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation, UpSampling2D
from tensorflow.keras.layers import  MaxPool2D, BatchNormalization, Reshape, Conv2DTranspose, LeakyReLU, Rescaling
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from PIL import Image, ImageEnhance, ImageOps
from tqdm import tqdm_notebook as tqdm

os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

Image.MAX_IMAGE_PIXELS = 10000000000
%matplotlib inline

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
tf.__version__

In [ ]:
####################

In [ ]:
## Seeding ##
np.random.seed(0)
tf.random.set_seed(0)

IMG_SIZE = 256
BATCH_SIZE = 256
WORKERS = 32
Channel = 3
latent_dim = 256

In [ ]:
train_dir = 'train_dir'
val_dir = 'val_dir'

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels=None,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=123,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels=None,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=123,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False)

In [ ]:
plt.figure(figsize=(10, 10))
for images in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        #plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
###########################

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)
normalized_train_ds = train_ds.map(lambda x: (normalization_layer(x), normalization_layer(x)))
normalized_val_ds = val_ds.map(lambda x: (normalization_layer(x), normalization_layer(x)))

In [ ]:
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, Channel), name="inputs")
x = inputs

In [ ]:
x = Conv2D(32, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(16, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(8, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(8, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
temp = MaxPool2D((2, 2))(x)

In [ ]:
x = Flatten()(temp)
units = x.shape[1]
encoded = Dense(latent_dim, name="latent")(x)
x = Dense(units)(encoded)
x = LeakyReLU(alpha=0.2)(x)
x = Reshape((temp.shape[1], temp.shape[2], temp.shape[3]))(x)

In [ ]:
x = Conv2DTranspose(8, (3, 3), strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2DTranspose(8, (3, 3), strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2DTranspose(16, (3, 3), strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2DTranspose(32, (3, 3), strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2DTranspose(Channel, (3, 3), strides=1, padding="same")(x)
#x = BatchNormalization()(x)
x = Activation("sigmoid", name="outputs")(x)

outputs = x

In [ ]:
autoencoder = Model(inputs, outputs)
autoencoder.summary()

In [ ]:
encoder = Model(inputs, encoded)
encoder.summary()

In [ ]:
#autoencoder.load_weights('weight.h5')
#autoencoder.weights
#encoder.weights

In [ ]:
# train AE #

In [ ]:
autoencoder.compile(tf.keras.optimizers.Adam(learning_rate=1e-3), loss=losses.MeanSquaredError())

In [ ]:
EPOCHS = 100

CSV_name = 'save.csv'
filepath = 'save.ep{epoch:02d}_vl{val_loss:.2f}.h5'

CALLBACKS = [tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                monitor="val_loss",
                                                verbose=0,
                                                save_best_only=False,
                                                save_weights_only=True,
                                                mode="auto",
                                                save_freq="epoch"),
             tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                              min_delta=0,
                                              patience=5,
                                              verbose=0,
                                              mode="auto",
                                              baseline=None,
                                              restore_best_weights=True),
             tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                  factor=0.3,
                                                  patience=3,
                                                  verbose=0,
                                                  mode="auto",
                                                  min_delta=0.0000001,
                                                  cooldown=0,
                                                  min_lr=0),
             tf.keras.callbacks.CSVLogger(CSV_name, separator=",", append=False)]


hist = autoencoder.fit(normalized_train_ds, 
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose='auto',
    callbacks=CALLBACKS,
    validation_split=0.0,
    validation_data=normalized_val_ds,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=BATCH_SIZE,
    validation_freq=1,
    max_queue_size=10,
    workers=32,
    use_multiprocessing=True)

In [ ]:
# encode #

In [ ]:
test_dir = 'test_data_dir'
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_center=False,
                                                              samplewise_center=False,
                                                              featurewise_std_normalization=False,
                                                              samplewise_std_normalization=False,
                                                              zca_whitening=False,
                                                              zca_epsilon=1e-6,
                                                              rotation_range=0,
                                                              width_shift_range=0,
                                                              height_shift_range=0,
                                                              brightness_range=None,
                                                              shear_range=0.0,
                                                              zoom_range=0.0,
                                                              channel_shift_range=0.0,
                                                              fill_mode='nearest',
                                                              cval=0.0,
                                                              horizontal_flip=False,
                                                              vertical_flip=False,
                                                              rescale=1./255,
                                                              preprocessing_function=None,
                                                              data_format=None,
                                                              validation_split=0.0,
                                                              dtype=None)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(IMG_SIZE, IMG_SIZE),
                                                 color_mode='rgb', shuffle=False, batch_size=BATCH_SIZE,
                                                 class_mode=None, interpolation='lanczos')

In [ ]:
image = next(iter(test_generator))
image

In [ ]:
test_filenames = test_generator.filepaths
test_filenames

In [ ]:
EncodedImages = encoder.predict(test_generator, workers=16, verbose=1)

In [ ]:
np.save('FileName.npy', test_filenames)
np.save('AE_EncodedImgs.npy', EncodedImages)